In [ ]:
import sys
sys.path.append('/home/samer/projects/fuzzy_sql/src')

from fuzzy_sql.fuzzy_sql import *
import json
import os
from pathlib import Path
import glob

In [ ]:
# set directories
root_dir = Path('/home/samer/projects/fuzzy_sql')
metadata_dir = os.path.join(root_dir, 'data/sdgd/processed/metadata')
real_dir = os.path.join(root_dir, 'data/sdgd/processed/real')
syn_dir = os.path.join(root_dir, 'data/sdgd/processed/synthetic')
db_path = os.path.join(root_dir, 'db/sdgd.db')

In [ ]:
# poplulate filenames from folder into a list
real_fnames=glob.glob(real_dir+'/*.csv')
real_fnames=[Path(real_fname).stem for real_fname in real_fnames]
real_fnames.sort()


# generate filenames
meta_jsons=[]
real_csvs=[]
for real_fname in real_fnames:
    meta_jsons.append(real_fname+'.json')
    real_csvs.append(real_fname+'.csv')
    


In [ ]:
# connect to db
conn = sqlite3.connect(db_path)

In [ ]:
# import real data into database
for real_csv,meta_json in zip(real_csvs,meta_jsons):
    #Import real
    data,_=prep_data_for_db(os.path.join(real_dir,real_csv)) # Note: This function  can be used to quickly generate metadata template, but this is not shown here.
    with open(os.path.join(metadata_dir, meta_json), 'r') as f:
        metadata=json.load(f)
    print(f"Processing {real_csv} and {meta_json} ")
    candidate_db_idx=get_vars_to_index(metadata,data)
    table_name=Path(real_csv).stem
    make_table(table_name, data, conn,indx_vars=candidate_db_idx)